<a href="https://colab.research.google.com/github/AngelTroncoso/Agentes_Gratis/blob/main/Construccion_de_agentes_Avanzados_en_WEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent

In [ ]:
import getpass
import os
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)

In [ ]:
search.invoke("que es openai o1-mini")

In [ ]:
!pip -qU install faiss-gpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("how to upload a dataset")[0]

In [ ]:
from langchain.tools.retriever import create_retriever_tool

txt = '''Busca información sobre LangSmith.
¡Para cualquier pregunta sobre LangSmith,
debes usar esta herramienta!'''

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    txt,)

In [ ]:
tools = [search, retriever_tool]

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content=
                            "What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
agent_executor.invoke({"input": "hi!"})

In [ ]:
agent_executor.invoke({"input": "Cómo ayuda langsmith testing?"})

In [ ]:
print(agent_executor.invoke({"input": "Como intalar LangSmith?"})['output'])

In [ ]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "Mi nombre es Bob", "chat_history": []})

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="Mi nombre es Bob"),
            AIMessage(content="Hola Bob! ¿En qué puedo ayudarte hoy?"),
        ],
        "input": "Cuál es mi nombre?",
    }
)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)

Resumen
¿Qué es un Agente en LangChain?
Un agente es un sistema que utiliza un modelo de lenguaje como núcleo para tomar decisiones sobre qué acciones ejecutar y qué entradas proporcionar a esas acciones. Estos agentes pueden realizar tareas más complejas y específicas que las que se limitan a la generación de texto, ya que son capaces de:

Interactuar con herramientas externas (APIs, bases de datos, web scraping, etc.).
Tomar decisiones basadas en el contexto y las acciones previas.
Restringir sus respuestas utilizando reglas, memoria de conversaciones previas o datos en tiempo real.
Los agentes permiten controlar partes de código o acceder a información en tiempo real, lo que los hace mucho más útiles en aplicaciones que requieren integración con sistemas externos.

Uso de Toolkits en Agentes
En LangChain, los Toolkits son una colección de herramientas que se pueden integrar con un agente para que este realice acciones específicas. Cada tool (herramienta) puede conectarse a un servicio externo o sistema y actuar como un intermediario entre el modelo de lenguaje y la información que se necesita.

En este proyecto, estamos utilizando Tabli Search, una herramienta para realizar búsquedas en internet en tiempo real. Esto permite que el agente obtenga información actualizada sobre cualquier tema que se necesite, lo cual es ideal para consultas dinámicas o casos donde el modelo por sí solo no tenga acceso a datos específicos.

Ejemplo: Si el agente necesita obtener información reciente sobre un modelo de OpenAI, como el O1-Mini, puede utilizar Tabli Search para hacer la consulta, buscar en fuentes externas, y devolver una respuesta basada en los resultados encontrados.

Cómo Funcionan los Agentes con Múltiples Herramientas
En este caso, hemos combinado el uso de Tabli Search con un Retriever Tool. Esta segunda herramienta utiliza una base vectorial para almacenar y recuperar documentos fragmentados (como páginas web o PDFs) que ya hemos cargado y vectorizado.

El flujo general es el siguiente:

Input del Usuario: El agente recibe una consulta del usuario.
Elección de Herramienta: Basado en la naturaleza de la pregunta, el agente decide si utilizar su base de conocimientos local (almacenada en la base vectorial) o realizar una búsqueda en la web usando Tabli Search.
Búsqueda y Respuesta: Si es necesario, el agente hace la búsqueda y genera una respuesta utilizando las herramientas seleccionadas.
Respuesta Final: El modelo de lenguaje, integrado con las herramientas, procesa los resultados y genera una respuesta para el usuario.
Este enfoque permite que el agente sea más versátil y útil, pudiendo manejar tanto información almacenada previamente como datos obtenidos en tiempo real.

Integración de la Base Vectorial
La base vectorial permite al agente almacenar y buscar en documentos previamente vectorizados (convertidos en representaciones numéricas llamadas embeddings). Cada vez que un usuario hace una pregunta, el agente puede buscar en esta base vectorial para recuperar información relevante, que luego se procesa junto con los resultados de las búsquedas en internet.

Vectorización: Este proceso convierte los documentos (como PDFs o sitios web) en vectores, que son representaciones numéricas del contenido semántico. Esto permite realizar búsquedas por similitud en los documentos cargados. Los vectores similares se encuentran cerca en el espacio vectorial, lo que permite recuperar información relevante rápidamente.

Construcción del Prompt para el Agente
En este proyecto, el agente tiene un prompt personalizado que define cómo debe comportarse. El System Prompt indica las reglas que el agente debe seguir, como consultar los documentos o utilizar Tabli Search para información externa. Este prompt también define cómo debe interactuar con las diferentes herramientas disponibles y cómo estructurar las respuestas para los usuarios.

